# RAG (Retrieval Augmented Generation) using LangChain and Chroma

## Data generation for Knowledge base

- Data chunking: [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)
    - 데이터 chunk 단위로 split하여 LLM에 query를 전달하면, 토큰 절약 가능
    - 생성 시간 감소
    - LangChain에서는 다양한 Text splitter들을 제공
- `chunk_size`: split된 chunk 하나의 크기
- `chunk_overlap`: 각 chunk들의 중복되는 부분

### Example code

- [근로기준법](https://www.law.go.kr/%EB%B2%95%EB%A0%B9/%EA%B7%BC%EB%A1%9C%EA%B8%B0%EC%A4%80%EB%B2%95)에 관한 문서
    - 경험적으로, `doc` 파일 포맷이 가장 RAG에 적합
- 이를 기반으로, RAG를 구성

In [17]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("./labor.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

### Huggingface Embedding 활용

In [18]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

embedding

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

# Huggingface Embedding의 성능이 제대로 안나온다면?

- 한국어 중에서는 Upstage Embedding이 가장 효과적
- 참고 [랭체인노트: Upstage Embedding](https://wikidocs.net/253106)

# 모델 정보

| Model                        | Release date | Context Length | Description                                                                 |
|------------------------------|--------------|----------------|-----------------------------------------------------------------------------|
| solar-embedding-1-large-query | 2024-05-10   | 4000           | 4K 컨텍스트를 지원하는 Solar-base Query Embedding 모델입니다. 이 모델은 검색 및 재정렬과 같은 정보 탐색 작업에서 사용자의 질문을 임베딩하는 데 최적화되어 있습니다. |
| solar-embedding-1-large-passage | 2024-05-10   | 4000           | 4K 컨텍스트를 지원하는 Solar-base Passage Embedding 모델입니다. 이 모델은 검색 문서나 텍스트를 임베딩하는 데 최적화되어 있습니다. |

In [3]:
# solar-embedding-1-large-passage를 사용해보자
import os
from langchain_upstage import UpstageEmbeddings
from dotenv import load_dotenv

load_dotenv()

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")

# 문장 전용 임베딩 모델
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage", api_key=UPSTAGE_API_KEY)

In [4]:
from langchain_chroma import Chroma

# 처음 데이터를 생성할 때
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-labor', persist_directory="./chroma")

# 데이터베이스에 추가할 때
# database = Chroma(collection_name='chroma-labor', persist_directory="./chroma", embedding_function=embedding)


In [5]:
query = '근로 기준법에 폭행 금지 조항이 있나요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [6]:
retrieved_docs

[Document(id='d7ee90e3-94c0-49f7-bf8b-08dcdf9d7c3a', metadata={'source': './labor.docx'}, page_content='제4조(근로조건의 결정) 근로조건은 근로자와 사용자가 동등한 지위에서 자유의사에 따라 결정하여야 한다.\n\n\n\n제5조(근로조건의 준수) 근로자와 사용자는 각자가 단체협약, 취업규칙과 근로계약을 지키고 성실하게 이행할 의무가 있다.\n\n\n\n제6조(균등한 처우) 사용자는 근로자에 대하여 남녀의 성(性)을 이유로 차별적 대우를 하지 못하고, 국적ㆍ신앙 또는 사회적 신분을 이유로 근로조건에 대한 차별적 처우를 하지 못한다.\n\n\n\n제7조(강제 근로의 금지) 사용자는 폭행, 협박, 감금, 그 밖에 정신상 또는 신체상의 자유를 부당하게 구속하는 수단으로써 근로자의 자유의사에 어긋나는 근로를 강요하지 못한다.\n\n\n\n제8조(폭행의 금지) 사용자는 사고의 발생이나 그 밖의 어떠한 이유로도 근로자에게 폭행을 하지 못한다.\n\n\n\n제9조(중간착취의 배제) 누구든지 법률에 따르지 아니하고는 영리로 다른 사람의 취업에 개입하거나 중간인으로서 이익을 취득하지 못한다.\n\n\n\n제10조(공민권 행사의 보장) 사용자는 근로자가 근로시간 중에 선거권, 그 밖의 공민권(公民權) 행사 또는 공(公)의 직무를 집행하기 위하여 필요한 시간을 청구하면 거부하지 못한다. 다만, 그 권리 행사나 공(公)의 직무를 수행하는 데에 지장이 없으면 청구한 시간을 변경할 수 있다.\n\n\n\n제11조(적용 범위) ① 이 법은 상시 5명 이상의 근로자를 사용하는 모든 사업 또는 사업장에 적용한다. 다만, 동거하는 친족만을 사용하는 사업 또는 사업장과 가사(家事) 사용인에 대하여는 적용하지 아니한다.\n\n② 상시 4명 이하의 근로자를 사용하는 사업 또는 사업장에 대하여는 대통령령으로 정하는 바에 따라 이 법의 일부 규정을 적용할 수 있다.\n\n③ 이 법을 적용하는 경우에 상시 사용하는 근로자 수

# Huggingface 모델을 로컬로 다운받아 실행하는 방법

In [2]:
import os

hf_models = os.path.join(os.getcwd(), "hf_models")
if not os.path.exists(hf_models):
    os.makedirs(hf_models)

In [3]:
import os
os.environ['HF_HOME'] = 'hf_models' # llm 모델을 다운로드할 경로

In [ ]:
from langchain.llms import HuggingFacePipeline

# HuggingFace Model ID
model_id = 'OpenGVLab/InternVL2_5-26B'

# HuggingFacePipeline 객체 생성
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id, 
    device=2,               # -1: CPU(default), 0번 부터는 CUDA 디바이스 번호 지정시 GPU 사용하여 추론
    task="text-generation", # 텍스트 생성
    trust_remote_code=True,
    model_kwargs={"temperature": 0.2, "max_length": 2048}, # 모델 파라미터
    pipeline_kwargs={"max_tokens": 2048} # 파이프라인 파라미터
)

ValueError: Loading OpenGVLab/InternVL-Chat-V1-5 requires you to execute the configuration file in that repo on your local machine. Make sure you have read the code there to avoid malicious use, then set the option `trust_remote_code=True` to remove this error.

In [7]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="phi4-14B")

/tmp/ipykernel_4075806/2145504416.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="phi4-14B")


In [20]:
response = llm.invoke("근로기준법애 대해서 알고있는지??")
print(response.content)

네, 근로기준법에 대해 알고 있습니다. 근로기준법은 한국에서 근로자의 권익을 보호하고 고용주와 근로자 간의 공정한 노동관계를 조성하기 위해 마련된 법률입니다. 이 법은 근로자의 근로조건, 보수, 휴게, 안전 및 보건 등에 대한 규정을 포함하고 있습니다. 주요 내용으로는 다음과 같은 것들이 있습니다.

1. **근로시간 및 휴게**: 근로시간의 최대치, 휴게시간, 주 1회 24시간 이상의 휴식 등이 규정되어 있습니다.
2. **임금**: 최저임금의 설정, 임금 지급 방법, 지연 지급 금지 등에 대한 규정이 포함되어 있습니다.
3. **휴가**: 연차휴가, 임신·출산 휴가, 병가 등에 대한 규정이 있습니다.
4. **노동안전보건**: 작업환경의 안전과 근로자의 건강을 보호하기 위한 규정이 포함되어 있습니다.
5. **노동계약**: 노동계약 체결, 해지 및 해고 절차에 대한 규정이 있습니다.
6. **노동조합 및 단체교섭**: 노동조합의 결성과 활동, 단체교섭에 대한 규정이 포함되어 있습니다.

이 법은 근로자와 고용주 간의 권리와 의무를 명확히 하고, 근로자의 권익을 보호하며, 사회적 공정과 경제적 발전을 도모하는 데 기여합니다.


In [22]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/opt/conda/envs/chatbot/lib/python3.10/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
prompt

TypeError: list indices must be integers or slices, not str

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

ValidationError: 1 validation error for LLMChain
prompt
  Input should be a valid dictionary or instance of BasePromptTemplate [type=model_type, input_value='문서를 참고해서 답변', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

In [11]:
# 아래 구문이 실행되기 위해선 반드시 ollama 서버가 실행되고 있어야 합니다.
ai_message = qa_chain({"query": query})

/tmp/ipykernel_4075806/3627372662.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [13]:
ai_message

{'query': '근로 기준법에 폭행 금지 조항이 있나요?',
 'result': "The South Korean Labor Standards Act sets forth regulations to ensure fair labor practices and protect the rights of workers. Here's a summary of the key provisions:\n\n1. **Purpose**: The Act aims to establish basic labor conditions to ensure workers' fundamental livelihoods, improve their quality of life, and contribute to balanced national economic development.\n\n2. **Definitions**:\n   - **Worker (근로자)**: Anyone providing labor for wages, regardless of job type, to a business or workplace.\n   - **Employer (사용자)**: Business owners or managers, and others acting on behalf of employers regarding workers.\n   - **Labor (근로)**: Includes both mental and physical work.\n   - **Labor Contract (근로계약)**: An agreement where workers provide labor and employers pay wages.\n   - **Wages (임금)**: All payments made by employers to workers for their labor, regardless of the form of payment.\n   - **Average Wage (평균임금)**: Calculated based on the total